In [1]:

import pandas as pd
import numpy as np 
import nltk
import string
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nltk
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc,accuracy_score,f1_score,precision_score,recall_score
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import precision_score,recall_score,auc

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
train_final = pd.read_csv("train_final_featured.csv")

In [4]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347469 entries, 0 to 347468
Data columns (total 50 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   building_id                             347469 non-null  int64  
 1   geo_level_1_id                          347469 non-null  int64  
 2   geo_level_2_id                          347469 non-null  int64  
 3   geo_level_3_id                          347469 non-null  int64  
 4   count_floors_pre_eq                     347469 non-null  int64  
 5   age                                     347469 non-null  int64  
 6   area_percentage                         347469 non-null  int64  
 7   height_percentage                       347469 non-null  int64  
 8   land_surface_condition                  347469 non-null  object 
 9   foundation_type                         347469 non-null  object 
 10  roof_type                               3474

In [5]:
cat_feats = ['land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position','legal_ownership_status',
       'plan_configuration']

In [6]:
train_final = pd.get_dummies(train_final,columns=cat_feats,drop_first=False)
train_final = train_final.drop(columns ="prob1_geo1")
train_final = train_final.drop(columns ="prob2_geo1")
train_final = train_final.drop(columns ="prob3_geo1")
train_final = train_final.drop(columns ="prob1_geo2")
train_final = train_final.drop(columns ="prob2_geo2")
train_final = train_final.drop(columns ="prob3_geo2")
train_final = train_final.drop(columns ="prob1_geo3")
train_final = train_final.drop(columns ="prob2_geo3")
train_final = train_final.drop(columns ="prob3_geo3")

In [7]:
train = train_final[(train_final['damage_grade'] != -1)]
#train.drop('building_id',axis=1)
test = train_final[(train_final['damage_grade'] == -1)]
test.index -=260601

In [8]:
from sklearn.model_selection import train_test_split
X = train.drop('damage_grade',axis=1)
y = train['damage_grade']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=35)


In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [51]:
classifiers = [
    
    DecisionTreeClassifier(),
    RandomForestClassifier(),
   # GradientBoostingClassifier()
]

In [52]:
def model_and_test(X, y, classifiers):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 35)
    
    for model in classifiers:
        this_model = model.__class__.__name__
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        print(f'{this_model} f1 score:')
        score = f1_score(y_test, y_pred, average='micro')
        print(f'{score:.4f}')
        print('\n')

In [53]:
model_and_test(X, y, classifiers)

DecisionTreeClassifier f1 score:
0.6723


RandomForestClassifier f1 score:
0.7445




In [24]:
model_rf = RandomForestClassifier(max_depth=9, min_samples_split=5, n_estimators=500)
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)

print(f'{model_rf} f1 score:')
score = f1_score(y_test, y_pred, average='micro')
print(f'{score:.4f}')
print('\n')

RandomForestClassifier(max_depth=9, min_samples_split=5, n_estimators=500) f1 score:
0.7449




final score 0.7276

In [23]:
model_dt = DecisionTreeClassifier(max_depth=8, min_samples_split=5)
model_dt.fit(X_train, y_train)
y_pred = model_dt.predict(X_test)

print(f'{model_dt} f1 score:')
score = f1_score(y_test, y_pred, average='micro')
print(f'{score:.4f}')
print('\n')

DecisionTreeClassifier(max_depth=8, min_samples_split=5) f1 score:
0.7486




final 7273

In [34]:
model_gb = GradientBoostingClassifier(learning_rate=0.03,n_estimators=500,max_depth=8,min_samples_split=5)
model_gb.fit(X_train, y_train)
y_pred = model_gb.predict(X_test)

print(f'{model_gb} f1 score:')
score = f1_score(y_test, y_pred, average='micro')
print(f'{score:.4f}')
print('\n')
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


GradientBoostingClassifier(learning_rate=0.03, max_depth=8, min_samples_split=5,
                           n_estimators=500) f1 score:
0.7603


Accuracy: 0.7603284643524085


final 7419

In [30]:
#test = test.drop('damage_grade',axis=1)
prediction=model_gb.predict(test).tolist()

In [31]:
result=pd.DataFrame(prediction)

In [32]:
result['building_id']=test['building_id']
result.rename(columns={0:'damage_grade'},inplace=True)
result=result[['building_id','damage_grade']]
result.head()

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,3
3,745817,1
4,421793,3


In [33]:
result.to_csv('result.csv',index=False)